# Overview 7: SAS Macro Language

## Introduction

The SAS Macro Language allows:
- **Automation** of programs.
- **Passing parameters** between DATA and PROC steps (also applies to PROC SQL, though not covered here).

The macro language consists of:
- **Macro variables** (`&table`)
- **Macro instructions** (`%let`)
- **Macro functions** (`%scan(...)`)
- **Macro programs**: sequences of macro instructions and functions

---

## 1) Macro Variables

### Declaration
Macro variables are declared using the `%let` statement :

In [ ]:
%let ds = tablesas;                 /* A SAS table */
%let path = c:\users;              /* A file path */
%let var1 = poids;                  /* A variable */
%let var2 = taille;                  /* Another variable */
%let varlist = sexe age taille poids;            /* A list of variables */

### Referencing a Macro Variable
Use `&` to reference a macro variable :

In [ ]:
proc plot data = &ds;
  plot &var1 * &var2;
  title "Analysis of &ds";
  title2 "For variables &varlist";
run;

### SAS Resolves the Program as :

In [ ]:
proc plot data = tablesas;
  plot poids * taille;
  title "Analysis of tablesas";
  title2 "For variables sexe age taille poids";
run;

### Notes:
- Macro variables are replaced by their values before execution.
- Macro variables inside single quotes are **not resolved**.
- Modifying macro variables applies changes dynamically.

### Macro Variable Rules:
- Names must start with a letter or `_`.
- Max name length: **32 characters**.
- Max content length: **65534 characters**.
- `%put &ds;` prints macro variable values in the log.

### Special Resolution Rules:
| Syntax | Resolution |
|--------|------------|
| `&x&y` | Concatenates `x` and `y` values |
| `&x.y` | Concatenates `x` value with string `y` |
| `&&var` | Replaces `&&` with `&` before resolving |

---

## 2) Macro Instructions

### `%let` Example:

In [ ]:
%let country = France - Germany - Italy;
%put &country;

**Output:**

In [ ]:
France - Germany - Italy

### `%eval` for Calculations

In [ ]:
%let amount = 5+7;
%put &amount;   /* Output: 5+7 */
%let result = %eval(&amount);
%put &result;   /* Output: 12 */

## 3) Macro Programs

### Open Code Macros

In [ ]:
%let ds = tablesas;
%let var1 = poids;
%let var2 = taille;

proc plot data = &ds;
  plot &var1 * &var2;
  title "Analysis of &ds";
run;

### True Macro Programs

In [ ]:
%macro loop;
%do i = 1 %to 10;
  proc print data=tab&i;
  title "Example Loop";
  run;
%end;
%mend loop;

%loop;

**Generated Code :**

In [ ]:
proc print data=tab1; run;
proc print data=tab2; run;
...
proc print data=tab10; run;

### Macros with Positional Parameters

In [ ]:
%macro print_table(table, var1, var2);
  proc print data=&table;
    var &var1 &var2;
    title "Example with Positional Parameters";
  run;
%mend;

%print_table(tab1, name, age);

### Macros with Keyword Parameters

In [ ]:
%macro print_table(table=, vars=);
  proc print data=&table;
    var &vars;
    title "Example with Keyword Parameters";
  run;
%mend;

%print_table(table=tab1, vars=age poids);

## 4) Example Summary

Create a macro that
1. Displays table contents.
2. Prints the first N observations.
3. Creates a subset of the data based on year.
4. Uses macro parameters dynamically.

In [ ]:
%macro info(table, year, obs);
  proc contents data=in.&table;
  title "Table: &table";
  run;

  proc print data=in.&table (obs=&obs);
  title "First &obs observations of &table";
  run;

  data &table&year;
    set in.&table;
    if an = &year;
  run;

  proc print data=&table&year;
  title "Subset &table&year";
  run;
%mend;

%info(prod1, 94, 10);

## 5) Macro Functions

### `%index`: Find Position of a Substring

In [ ]:
%let countries = France - Germany - Italy - Spain;
%let pos = %index(&countries, Germany);
%put &pos; /* Output: 10 */

### `%length`: Get Length of a Macro Variable

In [ ]:
%let length = %length(&countries);
%put &length; /* Output: 31 */

### `%substr`: Extract Part of a String

In [ ]:
%put %substr(&countries, 8, 7); /* Output: Germany */

### `%upcase`: Convert to Uppercase

In [ ]:
%put %upcase(&countries); /* Output: FRANCE - GERMANY - ITALY - SPAIN */

### `%eval`: Perform Integer Calculations

In [ ]:
%let i = 1;
%let j = &i + 2;
%let k = %eval(&i + 9);
%put &j; /* Output: 1+2 */
%put &k; /* Output: 10 */